In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Dados

### Função para fazer os plots

In [ ]:
def plotar


In [ ]:
# Peso Vazio x MTOW

In [ ]:
# Número de passageiros x MTOW

In [ ]:
# Número de passageiros x Comprimento de fuselagem